# Capstone: Recommender System
## 3. Content Based Recommender System

In [1]:
from nltk import ngrams
import regex as re
import requests
import pandas as pd
import time
import random

import scikitplot as skplt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
df = pd.read_csv('../datasets/product_info_clean.csv')

In [3]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281 entries, 0 to 280
Data columns (total 14 columns):
full_name           281 non-null object
brand               281 non-null object
name                281 non-null object
Category            281 non-null object
URL                 281 non-null object
rating              281 non-null float64
price               281 non-null float64
productsize         281 non-null object
oz                  281 non-null float64
price_oz            281 non-null float64
descriptions        281 non-null object
ingredients         281 non-null object
pure_ingredients    281 non-null object
pic                 281 non-null object
dtypes: float64(4), object(10)
memory usage: 30.8+ KB


In [4]:
#create 
df1 = df[['full_name','Category','pure_ingredients']]

In [5]:
#cleaning the ingredients by remove some characters
for i in range(len(df1.pure_ingredients)):
    a = df1.pure_ingredients[i]
    old = ['(',')','  ']
    for char in old:
        a = a.replace(char,' ')
    df1.pure_ingredients[i] = a
    

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
#create bag-of-word
#column for unigram and column for combitation of unigram and bigram
df1.loc[:,'unigram'] = np.nan
df1.loc[:,'uni_bigram'] = np.nan

In [8]:
df1.head()

,full_name,Category,pure_ingredients,unigram,uni_bigram
0,FRESH Soy Face Cleanser,face-wash-facial-cleanser,"Water, Coco-Glucoside, Glycerin, Butylene Glyc...",NaN,NaN
1,YOUTH TO THE PEOPLE Superfood Antioxidant Clea...,face-wash-facial-cleanser,"Water, Sodium Cocoyl Glutamate, Cocamidopropyl...",NaN,NaN
2,FARMACY Green Clean Makeup Removing Cleansing ...,face-wash-facial-cleanser,"Cetyl Ethylhexanoate, Caprylic/Capric Triglyce...",NaN,NaN
3,FRESH Sugar Strawberry Exfoliating Face Wash,face-wash-facial-cleanser,"Glycerin, Sucrose, Butylene Glycol, Fragaria V...",NaN,NaN
4,PHILOSOPHY Purity Made Simple Cleanser,face-wash-facial-cleanser,"Water, Sodium Lauroamphoacetate, Sodium Tridec...",NaN,NaN


In [1041]:
#method 1
#unigram
for i in range(len(df1.pure_ingredients)):
    ingredients_tokens = tokenizer.tokenize(df1.pure_ingredients[i].lower())

    df1.unigram[i] = ingredients_tokens
    

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:1015: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value


In [1042]:
#method 2
#combination of uigram and bigram

for i in range(len(df1.pure_ingredients)):
    ingredients_tokens = df1.pure_ingredients[i].lower().split(', ')
    l = []
    for token in ingredients_tokens:
        if ' ' in token:
            bigrams = ngrams(token.split(), 2)
            l.extend(bigrams)
        else:
            l.append(token)
    df1.uni_bigram[i] = l
    

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [1043]:
df1.head()

,full_name,Category,pure_ingredients,unigram,uni_bigram
0,FRESH Soy Face Cleanser,face-wash-facial-cleanser,"Water, Coco-Glucoside, Glycerin, Butylene Glyc...","[water, coco, glucoside, glycerin, butylene, g...","[water, coco-glucoside, glycerin, (butylene, g..."
1,YOUTH TO THE PEOPLE Superfood Antioxidant Clea...,face-wash-facial-cleanser,"Water, Sodium Cocoyl Glutamate, Cocamidopropyl...","[water, sodium, cocoyl, glutamate, cocamidopro...","[water, (sodium, cocoyl), (cocoyl, glutamate),..."
2,FARMACY Green Clean Makeup Removing Cleansing ...,face-wash-facial-cleanser,"Cetyl Ethylhexanoate, Caprylic/Capric Triglyce...","[cetyl, ethylhexanoate, caprylic, capric, trig...","[(cetyl, ethylhexanoate), (caprylic/capric, tr..."
3,FRESH Sugar Strawberry Exfoliating Face Wash,face-wash-facial-cleanser,"Glycerin, Sucrose, Butylene Glycol, Fragaria V...","[glycerin, sucrose, butylene, glycol, fragaria...","[glycerin, sucrose, (butylene, glycol), (fraga..."
4,PHILOSOPHY Purity Made Simple Cleanser,face-wash-facial-cleanser,"Water, Sodium Lauroamphoacetate, Sodium Tridec...","[water, sodium, lauroamphoacetate, sodium, tri...","[water, (sodium, lauroamphoacetate), (sodium, ..."


In [1044]:
#read in chemicals file
#chemicals file consists various chemicals that give products different effects
df2 = pd.read_csv('../datasets/chemicals.csv', encoding='latin1')
df2.feature.value_counts()

moisturising effect    213
antioxidant            158
soothing               120
cleanliness             93
viscosity               89
whitening               61
fragrance               54
irritancy               44
Name: feature, dtype: int64

In [1045]:
#create feature list
feature_list = ['moisturising effect','cleanliness','antioxidant','whitening','soothing','irritancy','viscosity','fragrance']
df_feature = pd.DataFrame({'feature':feature_list,'chemicals_uni':'','chemicals_unibi':''}) 

In [1046]:
#method 1 unigram
for i in range(len(feature_list)):
    chem = df2[df2.feature == feature_list[i]].chemicals
    l = []
    for c in chem:
        l=l+tokenizer.tokenize(c.lower())

        df_feature.chemicals_uni[i] = l
    
    

In [1047]:
#method 2 combination of unigram and bigram
for i in range(len(feature_list)):
    chem = df2[df2.feature == feature_list[i]].chemicals
    l = []
    for c in chem:
        if ' ' in c:
            bigrams = ngrams(c.lower().split(), 2)
            l.extend(bigrams)
        else:
            l.append(c.lower())
    df_feature.chemicals_unibi[i] = l



In [1048]:
df_feature

,feature,chemicals_uni,chemicals_unibi
0,moisturising effect,"[acetamide, mea, agarose, ammonium, lactate, a...","[(acetamide, mea), agarose, (ammonium, lactate..."
1,cleanliness,"[glycolic, acid, lactic, acid, apple, extract,...","[(glycolic, acid), (lactic, acid), (apple, ext..."
2,antioxidant,"[alfalfa, extract, alpha, lipoic, acid, alpha,...","[(alfalfa, extract), (alpha, lipoic), (lipoic,..."
3,whitening,"[lactic, acid, angelica, apple, extract, apric...","[(lactic, acid), angelica, (apple, extract), (..."
4,soothing,"[acetamide, mea, alchemilla, extract, allantoi...","[(acetamide, mea), (alchemilla, extract), alla..."
5,irritancy,"[alcohol, acetone, aluminum, acetate, solution...","[alcohol, acetone, (aluminum, acetate), (aceta..."
6,viscosity,"[bees, wax, bentonite, benzoyl, alcohol, butyl...","[(bees, wax), bentonite, (benzoyl, alcohol), (..."
7,fragrance,"[amyl, cinnamal, anise, extract, r, anisic, ac...","[(amyl, cinnamal), (anise, extract), (r-anisic..."


In [1050]:
#jaccard_similarity
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return len(s1.intersection(s2)) / len(s1.union(s2))


In [1097]:
#using different method for different features
feature_list_1 = ['moisturising effect','whitening','viscosity']
feature_list_2 = ['cleanliness','antioxidant','fragrance']
def ingre_similarity(product, feature, chemical):
    for e in feature:
        s = []
        length = len(product)
        n = df_feature[df_feature['feature'] == e].index.values[0]
        for i in range(length):
            
            score1 = jaccard_similarity(product[i][0:int(length/3)], chemical[n])  
            score2 = jaccard_similarity(product[i][int(length/3):int(length*2/3)], chemical[n])
            score3 = jaccard_similarity(product[i][int(length*2/3):length], chemical[n])
            score = 0.5*score1 + 0.3*score2 +0.2*score3   
            s.append(score)
        df1[e] = s
        
ingre_similarity(df1.unigram, feature_list_1, df_feature.chemicals_uni)
ingre_similarity(df1.uni_bigram, feature_list_2, df_feature.chemicals_unibi)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [1054]:
df1.sort_values(by='moisturising effect', ascending=False)[['full_name','Category','moisturising effect']].head(20)

,full_name,Category,moisturising effect
239,IT COSMETICS Confidence in a Cream Hydrating M...,moisturizer-skincare,0.074246
57,IT COSMETICS Bye Bye Makeup Cleansing Balm,face-wash-facial-cleanser,0.067510
242,LANEIGE Water Bank Moisture Cream,moisturizer-skincare,0.065564
39,IT COSMETICS Confidence in a Cleanser,face-wash-facial-cleanser,0.062180
227,IT COSMETICS Secret Sauce Anti-Aging Moisturizer,moisturizer-skincare,0.058900
228,SK-II R.N.A. POWER Face Cream,moisturizer-skincare,0.058804
276,DR. DENNIS GROSS SKINCARE Ferulic + Retinol An...,moisturizer-skincare,0.056190
0,FRESH Soy Face Cleanser,face-wash-facial-cleanser,0.055904
5,FRESH Soy Face Cleanser Limited Edition,face-wash-facial-cleanser,0.055904
260,ORIGINS GinZing™ SPF 40 Energy-Boosting Tinted...,moisturizer-skincare,0.055470


In [952]:
df1.sort_values(by='cleanliness', ascending=False)[['full_name','Category','cleanliness']].head()

,full_name,Category,cleanliness
65,JACK BLACK Pure Clean Daily Facial Cleanser,face-wash-facial-cleanser,0.085443
55,ORIGINS GinZing™ Refreshing Scrub Cleanser,face-wash-facial-cleanser,0.080357
25,BOSCIA Detoxifying Black Charcoal Cleanser,face-wash-facial-cleanser,0.078947
35,MURAD AHA/BHA Exfoliating Cleanser,face-wash-facial-cleanser,0.077206
70,NEOGEN DERMALOGY Green Tea Real Fresh Foam Cle...,face-wash-facial-cleanser,0.074897
84,ORIGINS Zero Oil™ Deep Pore Cleanser,face-wash-facial-cleanser,0.074627
92,KORRES Wild Rose Foaming Cream Cleanser,face-wash-facial-cleanser,0.069364
19,OLEHENRIKSEN Find Your Balance™ Oil Control Cl...,face-wash-facial-cleanser,0.068627
62,CLINIQUE Pep-Start 2-in-1 Exfoliating Cleanser,face-wash-facial-cleanser,0.067308
21,CLINIQUE Liquid Facial Soap,face-wash-facial-cleanser,0.067039


In [953]:
df1.sort_values(by='viscosity', ascending=False)[['full_name','Category','viscosity']].head(30)


,full_name,Category,viscosity
239,IT COSMETICS Confidence in a Cream Hydrating M...,moisturizer-skincare,0.323542
242,LANEIGE Water Bank Moisture Cream,moisturizer-skincare,0.311780
227,IT COSMETICS Secret Sauce Anti-Aging Moisturizer,moisturizer-skincare,0.293850
260,ORIGINS GinZing™ SPF 40 Energy-Boosting Tinted...,moisturizer-skincare,0.288373
41,AMOREPACIFIC Treatment Cleansing Foam,face-wash-facial-cleanser,0.254058
276,DR. DENNIS GROSS SKINCARE Ferulic + Retinol An...,moisturizer-skincare,0.236119
236,FIRST AID BEAUTY Hello FAB Coconut Skin Smooth...,moisturizer-skincare,0.233477
209,DR. JART+ Cicapair ™ Tiger Grass Color Correct...,moisturizer-skincare,0.231196
204,BELIF The True Cream Moisturizing Bomb,moisturizer-skincare,0.226667
196,BELIF The True Cream Aqua Bomb,moisturizer-skincare,0.226357


In [954]:
df1.sort_values(by='whitening', ascending=False)[['full_name','Category','whitening']].head(30)

,full_name,Category,whitening
260,ORIGINS GinZing™ SPF 40 Energy-Boosting Tinted...,moisturizer-skincare,0.199933
39,IT COSMETICS Confidence in a Cleanser,face-wash-facial-cleanser,0.188842
239,IT COSMETICS Confidence in a Cream Hydrating M...,moisturizer-skincare,0.172617
201,IT COSMETICS CC+ Cream Oil-Free Matte with SPF 40,moisturizer-skincare,0.169047
231,GLAMGLOW GLOWSTARTER™ Mega Illuminating Moistu...,moisturizer-skincare,0.163527
57,IT COSMETICS Bye Bye Makeup Cleansing Balm,face-wash-facial-cleanser,0.153116
70,NEOGEN DERMALOGY Green Tea Real Fresh Foam Cle...,face-wash-facial-cleanser,0.151621
250,ESTÉE LAUDER Revitalizing Supreme Global Anti-...,moisturizer-skincare,0.148295
212,SUNDAY RILEY Tidal Brightening Enzyme Water Cream,moisturizer-skincare,0.148167
246,IT COSMETICS Bye Bye Redness Neutralizing Colo...,moisturizer-skincare,0.137758


In [955]:
df1.sort_values(by='antioxidant', ascending=False)[['full_name','Category','antioxidant']].head(30)

,full_name,Category,antioxidant
70,NEOGEN DERMALOGY Green Tea Real Fresh Foam Cle...,face-wash-facial-cleanser,0.091072
246,IT COSMETICS Bye Bye Redness Neutralizing Colo...,moisturizer-skincare,0.072713
59,OLEHENRIKSEN Truth™ On the Glow Cleansing Cloths,face-wash-facial-cleanser,0.067925
227,IT COSMETICS Secret Sauce Anti-Aging Moisturizer,moisturizer-skincare,0.066365
233,YOUTH TO THE PEOPLE Superfood Hyaluronic Acid ...,moisturizer-skincare,0.066265
237,SUNDAY RILEY C.E.O Vitamin C Rich Hydration Cream,moisturizer-skincare,0.066176
57,IT COSMETICS Bye Bye Makeup Cleansing Balm,face-wash-facial-cleanser,0.062112
201,IT COSMETICS CC+ Cream Oil-Free Matte with SPF 40,moisturizer-skincare,0.059245
231,GLAMGLOW GLOWSTARTER™ Mega Illuminating Moistu...,moisturizer-skincare,0.058304
224,ORIGINS Dr. Andrew Weil For Origins™ Mega-Mush...,moisturizer-skincare,0.057180


In [1055]:
df1.sort_values(by='fragrance', ascending=False)[['full_name','Category','fragrance']].head(30)

,full_name,Category,fragrance
182,ERNO LASZLO Light Controlling Lotion,facial-toner-skin-toner,0.043750
73,TATA HARPER Purifying Pore Detox Cleanser,face-wash-facial-cleanser,0.023810
157,LANCÔME Tonique Radiance Clarifying Refining T...,facial-toner-skin-toner,0.023810
166,KIEHL'S SINCE 1851 Blue Astringent Herbal Lotion,facial-toner-skin-toner,0.022989
163,DERMALOGICA Breakout Clearing All Over Toner,facial-toner-skin-toner,0.022321
188,LA MER Crème de la Mer,moisturizer-skincare,0.020833
197,LA MER Limited Edition Blue Heart Crème de la Mer,moisturizer-skincare,0.020833
199,LA MER Crème de la Mer Mini,moisturizer-skincare,0.020833
121,SHISEIDO Benefiance WrinkleResist24 Balancing ...,facial-toner-skin-toner,0.018692
257,KIEHL'S SINCE 1851 Calendula Serum-Infused Wat...,moisturizer-skincare,0.018692


In [1056]:
##part 2 sentiment analysis for soothing and irritancy 
#in this case, we will treat soothing as positive and irritancy as negetive
#use combination of unigram and bigram
def simple_sentiment(ingredient):
    
    # positive/negative 
    index1 = df_feature[df_feature['feature'] == 'soothing'].index.values[0]
    index2 = df_feature[df_feature['feature'] == 'irritancy'].index.values[0]
    positive = df_feature.chemicals_unibi[index1]
    negative = df_feature.chemicals_unibi[index2]

    # Count 
    positive_count = sum([1 for i in ingredient if i in positive])
    negative_count = sum([1 for i in ingredient if i in negative])
    
    # Calculate Sentiment Percentage 
    # (Positive Count - Negative Count) / (Total Count)

    return round((positive_count - negative_count) / len(ingredient), 6)

##mildness score 　
l = []
for ingre in df1.uni_bigram:
    score = simple_sentiment(ingre)
    l.append(score)
df1.loc[:,'mildness'] = l 

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [1058]:
df1[['full_name','mildness']].sort_values(by = 'mildness',ascending = False).head()

,full_name,mildness
165,MARIO BADESCU Buffering Lotion,0.214286
102,MARIO BADESCU Drying Lotion,0.200000
152,BLITHE Vital Treatment Essence for Hydrating,0.198276
12,DRUNK ELEPHANT Slaai™ Makeup-Melting Butter Cl...,0.183333
28,GLOW RECIPE Blueberry Bounce Gentle Cleanser,0.169811


In [1061]:
df1.head()

,full_name,Category,pure_ingredients,unigram,uni_bigram,moisturising effect,whitening,viscosity,fragrance,cleanliness,antioxidant,mildness
0,FRESH Soy Face Cleanser,face-wash-facial-cleanser,"Water, Coco-Glucoside, Glycerin, Butylene Glyc...","[water, coco, glucoside, glycerin, butylene, g...","[water, coco-glucoside, glycerin, (butylene, g...",0.055904,0.035271,0.050855,0.003937,0.014045,0.015094,0.071429
1,YOUTH TO THE PEOPLE Superfood Antioxidant Clea...,face-wash-facial-cleanser,"Water, Sodium Cocoyl Glutamate, Cocamidopropyl...","[water, sodium, cocoyl, glutamate, cocamidopro...","[water, (sodium, cocoyl), (cocoyl, glutamate),...",0.023102,0.021898,0.027439,0.000000,0.009375,0.016327,0.108696
2,FARMACY Green Clean Makeup Removing Cleansing ...,face-wash-facial-cleanser,"Cetyl Ethylhexanoate, Caprylic/Capric Triglyce...","[cetyl, ethylhexanoate, caprylic, capric, trig...","[(cetyl, ethylhexanoate), (caprylic/capric, tr...",0.027428,0.024911,0.031177,0.011450,0.005348,0.012868,0.131579
3,FRESH Sugar Strawberry Exfoliating Face Wash,face-wash-facial-cleanser,"Glycerin, Sucrose, Butylene Glycol, Fragaria V...","[glycerin, sucrose, butylene, glycol, fragaria...","[glycerin, sucrose, (butylene, glycol), (fraga...",0.030000,0.014286,0.033742,0.009901,0.003185,0.008197,0.050000
4,PHILOSOPHY Purity Made Simple Cleanser,face-wash-facial-cleanser,"Water, Sodium Lauroamphoacetate, Sodium Tridec...","[water, sodium, lauroamphoacetate, sodium, tri...","[water, (sodium, lauroamphoacetate), (sodium, ...",0.026074,0.031447,0.029255,0.003937,0.008333,0.005556,0.045455


In [1070]:
#adding product information
product_score1 = df[['full_name','brand', 'Category','rating','price_oz']]
product_score2 = df1[['moisturising effect','cleanliness','mildness','antioxidant','whitening','viscosity','fragrance']]
product_score = pd.concat([product_score1, product_score2], axis = 1)



In [1071]:
product_score.head()

,full_name,brand,Category,rating,price_oz,moisturising effect,cleanliness,mildness,antioxidant,whitening,viscosity,fragrance
0,FRESH Soy Face Cleanser,FRESH,face-wash-facial-cleanser,4.4,7.450980,0.055904,0.014045,0.071429,0.015094,0.035271,0.050855,0.003937
1,YOUTH TO THE PEOPLE Superfood Antioxidant Clea...,YOUTH TO THE PEOPLE,face-wash-facial-cleanser,4.3,4.500000,0.023102,0.009375,0.108696,0.016327,0.021898,0.027439,0.000000
2,FARMACY Green Clean Makeup Removing Cleansing ...,FARMACY,face-wash-facial-cleanser,4.6,10.625000,0.027428,0.005348,0.131579,0.012868,0.024911,0.031177,0.011450
3,FRESH Sugar Strawberry Exfoliating Face Wash,FRESH,face-wash-facial-cleanser,4.5,7.857143,0.030000,0.003185,0.050000,0.008197,0.014286,0.033742,0.009901
4,PHILOSOPHY Purity Made Simple Cleanser,PHILOSOPHY,face-wash-facial-cleanser,4.5,3.000000,0.026074,0.008333,0.045455,0.005556,0.031447,0.029255,0.003937


In [1081]:
##normalize tha score to the range of 0-1
def norm(series):
    l = []
    for i in range(len(series)):
        n = (series[i] - series.min()) / (series.max() - series.min())
        l.append(n)
    series = l
        

In [1088]:
features = ['price_oz','moisturising effect','cleanliness','mildness','antioxidant','whitening','viscosity','fragrance']
for feature in features:
    series = product_score[feature]
    l = []
    for i in range(len(series)):
        n = (series[i] - series.min()) / (series.max() - series.min())
        l.append(n)
    product_score[feature] = l

In [1096]:
product_score.to_csv('../datasets/product_score.csv')

## Cosine similarity

In [1105]:
product_score = pd.read_csv('product_score.csv')

In [1108]:
product_score.drop(columns = 'Unnamed: 0', axis =1, inplace = True)

In [1120]:
content = product_score.iloc[:,4:11]

In [1122]:
content.head()

,price_oz,moisturising effect,cleanliness,mildness,antioxidant,whitening,viscosity
0,0.036590,0.746686,0.493134,0.445784,0.575174,0.568793,0.500569
1,0.019622,0.293673,0.329167,0.590362,0.622127,0.330948,0.251537
2,0.054841,0.353410,0.187760,0.679137,0.490325,0.384542,0.291296
3,0.038926,0.388935,0.111819,0.362650,0.312338,0.195567,0.318574
4,0.010997,0.334709,0.292593,0.345017,0.211696,0.500771,0.270853


In [1139]:
#Cosine similarity
sim_matrix = cosine_similarity(content)
content_sim = pd.DataFrame(sim_matrix, columns=content.index, index=content.index)


In [1141]:
content_sim.to_csv('../datasets/content_sim.csv')